# Imports

In [ ]:
%load_ext autoreload

In [ ]:
import os, sys
import holoviews as hv
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))
import hpc05

from IPython.core.magic import register_cell_magic
import ipywidgets as widgets
run_cluster = widgets.Checkbox(
    value=False,
    description='Check to run cluster',
    disabled=False
)
@register_cell_magic
def cluster_mark_checked(line, cell):
    if run_cluster.value is True:
        return line, cell
    else:
        return "Command not run. Check the box above to run."

In [ ]:
import adaptive
adaptive.notebook_extension()

import numpy as np
import scipy.constants
import cmath

import functools as ft

import sns_system, plotting_results
from distributed_sns import AggregatesSimulationSet as ASS
from distributed_sns import SimulationSet as SS

### Define constants

In [ ]:
constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

# Cluster setup

In [ ]:
run_cluster

In [ ]:
%%cluster_mark_checked
hpc05.kill_remote_ipcluster()
run_cluster.value = False

In [ ]:
%%cluster_mark_checked
client, dview, lview = hpc05.start_remote_and_connect(20, folder='~/two_dim_majoranas', timeout=180)
run_cluster.value = False

# Define and plot system

In [ ]:
params['k']*40e-3

In [ ]:
k = lambda n, phase: (n*np.pi - phase)/syst_pars['L_m']
params['hbar']**2*k(10, np.pi)**2/(2*params['m_eff'])

In [ ]:
syst_pars = {'L_down' :  200,
               'L_up' : 200,
               'L_m' : 150,
               'L_x' : 5,
               'a' :  5,
            'mu_from_bottom_of_spin_orbit_bands': False,
             'with_vlead' : True,
             'k_x_in_sc' : True}
 
_=plotting_results.plot_syst(syst_pars, sns_system.dummy_params)

# Define standard parameters

In [ ]:
params_raw= dict(g_factor_middle = 26,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 10.188482006771377,
                 alpha_middle = 20,
                 alpha_left = 0,
                 alpha_right = 0,
                 Delta_left = 2,
                 Delta_right = 2,
                 B = 1,
                 phase = 4,
                 T = 0.0,
                 V = 0.0)

params = dict(**constants,
              **params_raw)

# Bandstructures

In [ ]:
import kwant

In [ ]:
syst, hopping = sns_system.make_sns_system(**syst_pars)

In [ ]:
_bands = kwant.physics.Bands(syst.leads[1], params=params)
def bands(k):
    return _bands(k*syst_pars['a'])/params['Delta_left']

In [ ]:
def abs_min_log_loss(xs, xy_scale, data):
    from adaptive.learner.learner1D import default_loss
    data = {k: np.log(np.abs(v).min()) for k, v in data.items()}
    return default_loss(xs, xy_scale, data)

In [ ]:
learner = adaptive.Learner1D(bands, [-.2,.2], abs_min_log_loss)

In [ ]:
runner = adaptive.Runner(learner, ntasks=48)
runner.live_info()

In [ ]:
import topology
topology.get_pfaffian(syst, params)

In [ ]:
import formulas as f
f.fermi_wavenumber(params['mu'], 0.02)

In [ ]:
learner.plot()[-.2:.2, -1.5:1.5]

In [ ]:
vf = f.fermi_velocity(params['mu'], 0.02)
f.thouless_energy(vf, syst_pars['L_m'])